In [1]:
import pylabnet.hardware.ni_daqs.nidaqmx_card as nidaqmx
from pylabnet.utils.logging.logger import LogClient
import pylabnet.network.client_server.nidaqmx_card as nidaqmx_card_server
import pylabnet.network.client_server.staticline as staticline_server

import pylabnet.hardware.staticline.staticline as staticline
from pylabnet.network.core.generic_server import GenericServer

#  Instantiate Logger

In [2]:
# logger = LogClient(
#     host='140.247.189.82', 
#     port=36864, 
#     module_tag='DAQ Log Server'
# )

logger = LogClient(
    host='172.22.16.1', 
    port=40035, 
    module_tag='DAQ Log Server'
)

c:\users\brain\pylabnet\pylabnet\utils\helper_methods.py:941: ResourceWarning: unclosed file <_io.TextIOWrapper name='c:\\users\\brain\\pylabnet\\pylabnet\\configs\\network_configuration.json' mode='r' encoding='cp1252'>
  subnet = load_config('network_configuration')['subnet']


# Instantiate DAQ

In [5]:
device_name = 'PXI1Slot6_2'

daq = nidaqmx.Driver(device_name=device_name, logger=logger)

In [6]:
daq.get_ai_voltage('ai2', max_range = 10.0)

[2.3780943220817274]

In [9]:
device_name = 'PXI1Slot4_2'
daq_ao = nidaqmx.Driver(device_name=device_name, logger=logger)

In [12]:
daq_ao.set_ao_voltage('ao2', 1.5)

# Instanciate Staticline

In [95]:
daq.set_ao_voltage('ao0', 0)

In [10]:
# Let assume we have a shutter connected to the HDAWG DIO-pin 1 which 
# is open if the pin voltage is high and closed otherwise.

test_staticline = staticline.Driver(
    name='Laser Green',
    logger=logger,
    hardware_module=daq,
    ao_output='ao2',
    down_voltage=0,
    up_voltage=3.3,
) 

# Instantiate  Servers

In [35]:
# DAQ Server
daq_service = nidaqmx_card_server.Service()
daq_service.assign_module(module=daq)
daq_service.assign_logger(logger=logger)
daq_service_server = GenericServer(
    service=daq_service, 
    host='localhost', 
    port=17972
)

In [36]:
daq_service_server.start()

In [150]:
# DAQ Server
daq_ao_service = nidaqmx_card_server.Service()
daq_ao_service.assign_module(module=daq_ao)
daq_ao_service.assign_logger(logger=logger)
daq_ao_service_server = GenericServer(
    service=daq_ao_service, 
    host='localhost', 
    port=17974
)

In [151]:
daq_ao_service_server.start()

In [13]:
# Staticline server
staticline_service = staticline_server.Service()
staticline_service.assign_module(module=test_staticline)
staticline_service.assign_logger(logger=logger)
staticline_service_server = GenericServer(
    service=staticline_service, 
    host='localhost', 
    port=12986
)

In [14]:
staticline_service_server.start()